In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import numpy as np
from scipy import stats # Import the stats module from SciPy

from models import construct_model

In [9]:
# --- 2. Data Pipeline ---
def get_mnist_loaders(batch_size):
    """Prepares the MNIST train, validation, and test data loaders."""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_loader

In [3]:
def corrupt_images(images, corruption_fraction):
    """
    Corrupts a batch of images by REPLACING a fraction of pixels with random noise.
    Args:
        images (torch.Tensor): A batch of images (B, C, H, W).
        corruption_fraction (float): The fraction of pixels to corrupt (0.0 to 1.0).
    Returns:
        torch.Tensor: The batch of corrupted images.
    """
    if corruption_fraction == 0.0:
        return images

    corrupted_images = images.clone()
    batch_size, _, height, width = images.shape
    num_pixels_total = height * width
    num_pixels_to_corrupt = int(corruption_fraction * num_pixels_total)

    for i in range(batch_size):
        # Generate a random permutation of pixel indices to corrupt
        indices = torch.randperm(num_pixels_total, device=images.device)[:num_pixels_to_corrupt]

        # Convert flat indices to 2D indices
        row_indices = indices // width
        col_indices = indices % width

        # --- Generate random pixel values to replace the existing ones ---
        # Values are from a uniform distribution across the full [-1, 1] range.
        random_pixels = torch.rand(num_pixels_to_corrupt, device=images.device) * 2 - 1

        # --- Place the new random pixels into the image ---
        corrupted_images[i, 0, row_indices, col_indices] = random_pixels

    return corrupted_images

In [4]:
def analyze_with_individual_corruption(data_rater, data_loader, num_batches=5, current_step=None):
    """
    Analyzes the DataRater by corrupting each image with a continuous random
    fraction and creating a scatter plot of scores vs. corruption.
    """
    print(f"\n--- Analyzing with Continuous Individual Corruption over {num_batches} batches --- MetaStep: {current_step}")
    data_rater.eval()  # Set the model to evaluation mode

    # These master lists will store the results from all batches
    all_scores = []
    all_fractions = []

    # Loop over the specified number of batches
    for i, (batch_images, _) in enumerate(data_loader):
        if i >= num_batches:
            break

        batch_images = batch_images.to(DEVICE)
        batch_size = batch_images.size(0)

        individually_corrupted_batch = torch.zeros_like(batch_images)
        fractions_for_this_batch = []

        # Corrupt each image in the batch one by one
        for j in range(batch_size):
            # --- NEW: Generate a continuous random fraction between 0.0 and 1.0 ---
            frac = np.random.uniform(0.0, 1.0)
            fractions_for_this_batch.append(frac)

            # Corrupt the single image (note: we keep the batch dimension)
            original_image = batch_images[j:j+1]
            corrupted_image = corrupt_images(original_image, frac)
            individually_corrupted_batch[j] = corrupted_image

        # Get the model's scores for the entire batch of individually corrupted images
        with torch.no_grad():
            scores = data_rater(individually_corrupted_batch)
            weights = torch.softmax(scores, dim=0)

        # Append the results to our master lists
        all_scores.extend(weights.cpu().numpy())
        all_fractions.extend(fractions_for_this_batch)

    # --- Plotting and Regression Analysis ---

    plt.figure(figsize=(12, 8))

    # Scatter plot with continuous x-axis values (jitter is no longer needed)
    plt.scatter(all_fractions, all_scores, alpha=0.3, label='Individual Image Score')

    # Perform and plot the linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(all_fractions, all_scores)
    r_squared = r_value**2

    x_line = np.array([min(all_fractions), max(all_fractions)])
    y_line = slope * x_line + intercept

    plt.plot(x_line, y_line,
             color='red', linewidth=2,
             label=f'Linear Regression (R² = {r_squared:.3f})')

    plt.title('DataRater Score vs. Individual Image Corruption', fontsize=16)
    plt.xlabel('Fraction of Corrupted Pixels')
    plt.ylabel('Raw Score (Rating)')
    plt.grid(True, linestyle=':')
    plt.legend()
    plt.show()

In [5]:
def visualize_corruption(data_rater, data_loader, num_images=5):
    """
    Selects images, displays them with varying corruption, and overlays the
    DataRater's score on each image.
    """
    print("\n--- Visualizing Image Corruption with DataRater Scores ---")

    # Ensure the model is on the correct device and in evaluation mode
    data_rater.to(DEVICE).eval()

    # Get a single batch of images
    sample_images, _ = next(iter(data_loader))

    corruption_fractions = [0.0, 0.25, 0.5, 0.75, 1.0]

    fig, axes = plt.subplots(num_images, len(corruption_fractions), figsize=(12, num_images * 2.2))

    for i in range(num_images):
        original_image = sample_images[i:i+1]  # Keep batch dim

        for j, frac in enumerate(corruption_fractions):
            ax = axes[i, j]

            # Create the corrupted version
            corrupted_image_tensor = corrupt_images(original_image, frac)

            # --- Get the DataRater's score for this specific image ---
            with torch.no_grad():
                score = data_rater(corrupted_image_tensor.to(DEVICE)).item()

            # Prepare image for plotting (denormalize, etc.)
            img_to_plot = corrupted_image_tensor.squeeze().cpu().numpy()
            img_to_plot = img_to_plot * 0.5 + 0.5

            ax.imshow(img_to_plot, cmap='gray')
            ax.axis('off')

            # --- Add the score as text onto the image ---
            # The text is placed at coordinates (x=2, y=26) on the image
            txt = ax.text(2, 26, f"{score:.3f}",
                          color='white', fontsize=11, fontweight='bold')
            # Add a black outline to the text for high visibility
            txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='black')])

            if i == 0:
                ax.set_title(f"{frac*100:.0f}% Noise")

    fig.suptitle('Sample Images with DataRater Scores', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [10]:
test_loader = get_mnist_loaders(128)

In [12]:
def load_data_rater_from_checkpoint(checkpoint_path, device='cpu'):
    """
    Load DataRater model using the existing construct_model function.
    """
    # Create model using the same constructor as training
    model = construct_model('DataRater')
    
    # Load the saved state dictionary
    state_dict = torch.load(checkpoint_path, map_location=device)
    
    # Load the state dict into the model
    model.load_state_dict(state_dict)
    
    # Set to evaluation mode and move to device
    model.eval()
    model = model.to(device)
    
    return model

In [ ]:
checkpoint_path = "experiments/your_run_id/data_rater.pt"  # Update this path
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data_rater = load_data_rater_from_checkpoint(checkpoint_path, device)
analyze_with_individual_corruption(trained_data_rater, test_loader, num_batches=10)